In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time


In [4]:
# ID video YouTube dan URL
youtubeid = "i6IOiUi6IYY"
video_url = f"https://www.youtube.com/watch?v={youtubeid}"

In [5]:

profile_path = r'C:\Users\LENOVO\AppData\Local\Google\Chrome\User Data\Default'  # Ganti dengan path profil Chrome Anda

def initialize_driver_with_profile(profile_path):
    """Inisialisasi driver dengan profil Chrome yang sudah ada."""
    chrome_options = Options()
    chrome_options.add_argument(f"user-data-dir={profile_path}")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-first-run")
    chrome_options.add_argument("--remote-debugging-port=9222")
    chrome_options.add_argument("--enable-logging")
    chrome_options.add_argument("--v=1")  # Level logging dasar
    # service = Service(driver_path)
    # driver = webdriver.Chrome(service=service, options=chrome_options)
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def detect_and_skip_ad(driver):
    """Periksa iklan dan klik tombol 'Skip Ad' saat tersedia."""
    ad_present = False
    try:
        # Cek jika elemen iklan atau teks sponsor ada
        WebDriverWait(driver, 15).until(
            EC.presence_of_any_elements_located([
                (By.CLASS_NAME, "video-ads.ytp-ad-module"),               # Div dengan kelas iklan
                (By.CLASS_NAME, "ytp-ad-player-overlay-layout_")          # Overlay iklan
            ])
        )
        ad_present = True
        print("Iklan terdeteksi.")

        # Jika iklan terdeteksi, tunggu tombol "Skip Ad" dan klik jika ada
        skip_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "ytp-skip-ad-button"))
        )
        skip_button.click()
        print("Tombol 'Skip Ad' diklik.")
        
    except Exception as e:
        print("Tidak ada iklan atau tombol 'Skip Ad' tidak ditemukan.")
        print(f"Error: {e}")

    return ad_present

def get_all_youtube_comments(driver, max_comments=None):
    """Mengambil semua komentar dari video YouTube dengan scroll tanpa batas, termasuk penulis, waktu, suka, dan isi komentar."""
    comments_data = []
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    while True:
        # Scroll ke bawah untuk memuat komentar baru
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)  # Jeda untuk memuat komentar baru

        # Ambil elemen penulis, waktu, suka, dan komentar
        comment_elements = driver.find_elements(By.XPATH, '//*[@id="content-text"]')
        author_elements = driver.find_elements(By.XPATH, '//*[@id="author-text"]')
        time_elements = driver.find_elements(By.XPATH, '//*[@id="published-time-text"]')
        like_elements = driver.find_elements(By.XPATH, '//*[@id="vote-count-middle"]')
        
        # Loop melalui elemen dan tambahkan data ke dalam list
        for i in range(len(comments_data), len(comment_elements)):
            comment_text = comment_elements[i].text
            author_name = author_elements[i].text
            comment_time = time_elements[i].text
            likes = like_elements[i].text if i < len(like_elements) else "0"  # Beri nilai default "0" jika tidak ada likes
            
            if comment_text and author_name and comment_time:
                comments_data.append({
                    "Author": author_name,
                    "Timestamp": comment_time,
                    "Likes": likes,
                    "Comment": comment_text
                })

                # Batas maksimal komentar jika disetel
                if max_comments and len(comments_data) >= max_comments:
                    return comments_data

        # Cek tinggi baru halaman untuk melihat apakah ada yang baru dimuat
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:  # Berhenti jika tidak ada lagi yang dimuat
            break
        last_height = new_height

    return comments_data

def save_to_xlsx(comments_data, filename):
    """Menyimpan komentar ke file XLSX."""
    df = pd.DataFrame(comments_data)
    df.to_excel(filename, index=False)
    print(f"Komentar berhasil disimpan di {filename}")


In [7]:
def save_to_csv(comments_data, filename):
    """Menyimpan komentar ke file csv."""
    df = pd.DataFrame(comments_data)
    df.to_csv(filename, index=False)
    print(f"Komentar berhasil disimpan di {filename}")


In [6]:

# Inisialisasi WebDriver dengan profil Chrome
driver = initialize_driver_with_profile(profile_path)

# Akses URL video YouTube
driver.get(video_url)
time.sleep(5)  # Tunggu halaman dimuat

# Periksa iklan dan lewati jika ada
ad_present = detect_and_skip_ad(driver)

# Jika tidak ada iklan atau iklan sudah dilewati, mulai mengambil komentar
if not ad_present or ad_present:
    comments_data = get_all_youtube_comments(driver, max_comments=None)  # None untuk scroll tanpa batas
    save_to_xlsx(comments_data, filename=f"youtube_comments_{youtubeid}_123123.xlsx")

# Tutup driver setelah selesai
driver.quit()


Tidak ada iklan atau tombol 'Skip Ad' tidak ditemukan.
Error: module 'selenium.webdriver.support.expected_conditions' has no attribute 'presence_of_any_elements_located'
Komentar berhasil disimpan di youtube_comments_i6IOiUi6IYY.xlsx


In [10]:
df = pd.DataFrame(comments_data)
df

,Author,Timestamp,Likes,Comment
0,@holdtightadele8017,2 days ago,132,Thomas Partey… man has been bringing it lately.
1,@archanakhakase6357,2 days ago,76,Saka is the best he prove this everytime by d...
2,@kabiruddin434,2 days ago,948,That pass from Ben White to Saka was amazing.
3,@everythingarsenaltv,2 days ago,591,Saka and Partey were so so good man
4,@tggazard9840,2 days ago,54,""" Keep Saka's name out your mouth, dude is wo..."
...,...,...,...,...
648,@gunerarse449,2 days ago,,r
649,@Jack-eee,2 days ago,1,Corrupt league
650,@UR-CRISTIANO-r9b,1 day ago,,Like
651,@gunerarse449,2 days ago,,r


In [8]:
save_to_csv(comments_data, filename=f"youtube_comments_{youtubeid}_123123.xlsx")

Komentar berhasil disimpan di youtube_comments_i6IOiUi6IYY_123123.xlsx
